### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

#### Define parameters

In [4]:
vpc_name='fast-ai'

#### Get Existing VPC by tag name

In [5]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [23]:
instance_name = f'{vpc_name}-instance'
instance_type = 'c5.4xlarge'

In [24]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.315100'

In [25]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [26]:
launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 450

In [27]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 450,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-8c4288f4',
 'InstanceType': 'c5.4xlarge',
 'KeyName': 'aws-key-fast-ai',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-f60fca88'],
   'SubnetId': 'subnet-f056ff89'}]}

In [28]:
instance = create_spot_instance(instance_name, launch_specs, spot_price='1.0'); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-00373e6880a58e2b9
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@35.167.1.160


ec2.Instance(id='i-00373e6880a58e2b9')

In [ ]:
instance = get_instance(instance_name); instance

### SSH

In [29]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EFS

In [30]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [31]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [32]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [33]:
out, _ = run_command(client, efs_mount_cmd)

run_command returned: 



In [34]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt



### Upload kaggle keys

In [35]:
out, _ = run_command(client, 'mkdir ~/.kaggle')

run_command returned: 



In [36]:
kaggle_file = Path.home()/'.kaggle/kaggle.json'

In [37]:
upload_file(client, str(kaggle_file), '.kaggle/kaggle.json')

### Upload image resize

In [ ]:
upload_path = Path.cwd()/'upload_scripts/Resize_Images.ipynb'
upload_file(client, str(upload_path), '.')